# Covid-19 Global Analysis

## Virus
Coronavirus which affected 227,931 globally and is a cause of death for 9,311 is declared as a pandemic by WHO, it is referred to as COVID-19. Many of us do not know why it is termed as 2019-nCOV or COVID-19. It is new and hasn't been detected in humans before, Corona is a Latin word for crown, the core of genetic material is enveloped in a protein that resembles the crown. As it was detected at the end of the year 2019, WHO named is as 2019-nCOV, COVID-19.

## How does it spread?
COVID-19 is not airborne, it spreads from person to person through close contact or from droplets that are scattered when a person with the virus sneezes or coughs. According to the CDCTrusted Source, people who have the virus are most contagious when they’re showing symptoms of COVID-19.

## Symptoms
Infecting the respiratory system with symptoms including that of a common cold, caused by the rhinovirus, or, the flu caused by the influenza virus, There may also be aches and pains, fatigue and, in some cases, vomiting and diarrhoea, shortness of breath and it leads to respiratory complications like pneumonia. 

Below representations depicted are derived from the data provided by Johns Hopkins University.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import folium
import seaborn as sns
import warnings
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import HTML
from fbprophet import Prophet
from folium.plugins import HeatMapWithTime
warnings.filterwarnings('ignore')
%matplotlib inline

Datasets used for reference in analysis and plotting. 

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:


COVID19_line_list_data = pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv")
COVID19_open_line_list = pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv")
covid_data = pd.read_csv("../input/novel-corona-virus-2019-dataset/covid_19_data.csv",parse_dates=True)
time_series_covid_19_confirmed = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv")
time_series_covid_19_deaths = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv")
time_series_covid_19_recovered = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv")
img=mpimg.imread('../input/corona-local/China poll graphic.png')
geo_l = pd.read_csv("../input/world-coordinates/world_coordinates.csv")


In [ ]:
# geo_l[geo_l['Country']=='China']
geo_l = geo_l.append({'Code' : 'CN' 
                      , 'Country' : 'Mainland China'
                      ,'latitude' : '35.86166'
                      ,'longitude' : '104.195397'
                     } , ignore_index=True)

Data :Recent date.

In [ ]:
covid_data['ObservationDate'].max()

In [ ]:
country_CDR=pd.DataFrame(covid_data.groupby(['ObservationDate','Country/Region'],as_index=False)[['Confirmed','Deaths','Recovered']].sum())


In [ ]:
geo_w_d_CDR=country_CDR.merge(geo_l,left_on='Country/Region',right_on='Country')
#geo_w_d_CDR=geo_w_d_CDR.drop(['country','name'],axis=1)
#geo_w_d_CDR


## Cases and situation around the world:

Below map shows citizens of 112 countries that were affected and confirmed with COVID-19. China, Europe and Iran taking the max toll. Median age of first 100 identified/registered cases is above 40, it strongly impacted the working class of the country.

In [ ]:
# create map and display it
geo_w_m = folium.Map(location=[10, -20] ,zoom_start=2.3,tiles='OpenStreetMap')

for lat, lon, value, name in zip(geo_w_d_CDR['latitude'], geo_w_d_CDR['longitude'], geo_w_d_CDR['Confirmed'], geo_w_d_CDR['Country/Region']):
    folium.CircleMarker([lat, lon],
                        radius=value/5000,
                        popup = ('<strong>Country</strong>: ' + str(name).capitalize() + '<br>'
                                '<strong>Confirmed Cases</strong>: ' + str(value) + '<br>'),
                        color='gray',
                        
                        fill_color='gray',
                        fill_opacity=0.1 ).add_to(geo_w_m)
geo_w_m

Below is the map showing global deaths reported.
Click on the bubble to know deaths occured in the country.

In [ ]:
# create map and display it

deaths = geo_w_d_CDR.loc[geo_w_d_CDR['Deaths'] > 0]
geo_w_m_D = folium.Map(location=[0, -30] ,zoom_start=2.3,tiles='OpenStreetMap')

for lat, lon, value, name in zip(deaths['latitude'], deaths['longitude'], deaths['Deaths'], deaths['Country/Region']):
    folium.CircleMarker([lat, lon],
                        radius=value/150,
                        popup = ('<strong>Country</strong>: ' + str(name).capitalize() + '<br>'
                                '<strong>Reported Deaths</strong>: ' + str(value) + '<br>'),
                        color='#CD5C5C',
                        
                        fill_color='#CD5C5C',
                        fill_opacity=0.1 ).add_to(geo_w_m_D)
geo_w_m_D

Below is the map showing patients showing positive health signs and recovered cases. 
Measures adopted by the countries played a major role in decreasing the spread of the virus. Measures that are put in place,
* Restricting travel and movement
* Screening at airports
* Closed schools and colleges
* Controlled public events and gatherings
* Corporate offices implementing work from home.

Click on the bubble to know recoveries per country.

In [ ]:
recovered = geo_w_d_CDR.loc[geo_w_d_CDR['Recovered'] > 0]
geo_w_m_R = folium.Map(location=[10, -20] ,zoom_start=2.3,tiles='OpenStreetMap')

for lat, lon, count, country in zip(recovered['latitude'], recovered['longitude'], recovered['Recovered'], recovered['Country/Region']):
    folium.CircleMarker([lat, lon],
                        radius=8,
                        popup = ('<strong>Country</strong>: ' + str(country).capitalize() + '<br>'
                                '<strong>Recovered</strong>: ' + str(count) + '<br>'),
                        color='Green',
                        
                        fill_color='Green',
                        fill_opacity=0.1 ).add_to(geo_w_m_R)
geo_w_m_R

In [ ]:
confirmed_latl = time_series_covid_19_confirmed[["Province/State","Lat","Long","Country/Region"]]
df_temp = covid_data.copy()
df_temp['Country/Region'].replace({'Mainland China': 'China'}, inplace=True)
df_latlong = pd.merge(df_temp, confirmed_latl, on=["Country/Region", "Province/State"])

## Countries affected accross the world

In [ ]:
# # import ipywidgets
# # base_map = folium.Map(location=[10, -20] ,zoom_start=2.3,tiles='CartoDB Dark_Matter')
# # HeatMapWithTime(df_latlong[['Lat','Long']]
# #                 , radius=5
# #                 , index=df_latlong['ObservationDate']
# #                 , auto_play=True
# #                 , gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}
# #                 , min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map)
# # base_map

fig = px.density_mapbox(df_latlong,
                        
                        lat="Lat", 
                        lon="Long", 
                        hover_name="Country/Region", 
                        hover_data=["Confirmed","Deaths","Recovered"], 
                        animation_frame="ObservationDate",
                        color_continuous_scale="thermal",
                        radius=7, 
                        zoom=0,height=700)
fig.update_layout(title='Worldwide Corona Virus Cases Time Lapse - Confirmed, Deaths, Recovered',
                  font=dict(family="Courier New, monospace",
                            size=18,
                            color="#7f7f7f")
                 )
fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
#         'center': {'lon': -10, 'lat': 10},
        'style': "carto-darkmatter",
#         'center': {'lon': -20, 'lat': -20},
        'zoom':1
        
        
       
        }
)
fig.update_layout(autosize=False,width=600,
    height=800,mapbox_center_lon=82)


fig.show()

In [ ]:
covid_data_n=covid_data.drop(columns=['Last Update','Province/State'],axis=1)
covid_data_n.groupby(['Country/Region','ObservationDate']).sum()
country_t=pd.DataFrame(covid_data_n.groupby(['Country/Region'])['Confirmed'].sum()).sort_values(by=['Confirmed'],ascending=False)[:11]
# country_t.index

In [ ]:
covid_lines=covid_data[covid_data['Country/Region'].isin(country_t.index)]
covid_lines=covid_lines.drop(columns=['SNo'],axis=True)
t=pd.DataFrame(covid_lines.groupby(['Country/Region','ObservationDate'],as_index=False).sum())
Zeros = t[t.Confirmed == 0]
t=t.drop(Zeros.index)
t.sort_values("ObservationDate", axis = 0, ascending = True, 
                 inplace = True,) 

# covid_lines['ObservationDate'] = covid_lines['ObservationDate'].dt.date

In [ ]:
t_all=country_t=pd.DataFrame(t.groupby(['ObservationDate']).sum())
# t_all=t_all.drop(columns=['Country/Region'],axis=1)
# t_all=t_all.set_index('ObservationDate')




Current situation across the world can be summarised from the below chart. With case more tha 180K the situation is critical.

In [ ]:
f, ax = plt.subplots(1,1, figsize=(14,8))
k = sns.lineplot(data=t_all)
plt.xticks(rotation=90)
plt.title('Global Situation')
plt.xlabel('Date')
plt.ylabel('Cases')
plt.show()

With few identified in the dawn of 2020 and end of 2019, the cases were ever-increasing and creating an atmosphere of fear and isolation among countries and citizens. It is imperative to understand that a country identified recently with fewer cases than China is in a stage where  China was in, initially. With no fear of risk and improper implementation of the measures may impact an exponential  spread and affect the nations both economically and socially. 

Below is the trend for top 10 countries with Confirmed, Death and recovered cases. Considering the fact that COVID-19 has a strong Wuhan/China connections, China leads all the 3 trend reports.

In [ ]:
f, ax = plt.subplots(1,1, figsize=(14,8))
g = sns.lineplot(x='ObservationDate', y='Confirmed', hue='Country/Region', data=t)
plt.xticks(rotation=90)
plt.title('Top 10 Countries - Cases Confirmed By Date')
plt.show()  

In [ ]:
f, ax = plt.subplots(1,1, figsize=(14,8))
g = sns.lineplot(x='ObservationDate', y='Deaths', hue='Country/Region', data=t)
plt.xticks(rotation=90)
plt.title('Top 10 Countries - Deaths Confirmed By Date')

plt.show() 

In [ ]:
f, ax = plt.subplots(1,1, figsize=(14,8))
g = sns.lineplot(x='ObservationDate', y='Recovered', hue='Country/Region', data=t)
plt.xticks(rotation=90)
plt.title('Top 10 Countries - Recoveries Confirmed By Date')

plt.show() 

# Global Impacts

After 2008 COVID-19 has put the biggest fear among global economies. Preventive measures that are put in place is hurting the Travel and Tourism industry. China is the second-largest economy and the largest importer of the oil, with manufacturing and travel impacted, demand for oil has been reduced. On 09-Mar oil prices dipped as much as third of its value and further declined after US President restricting travel from Europe. To spur the demand OECD+ agreement had been disregarded by Saudi and Russia to increase oil production and flood markets with cheaper options. Markets took the shock and momentarily fell to the situation of 2008.

Below is an image poll taken by Reuters between 7-13 February said they expected China’s economic growth to slump to 4.5% in the first quarter of 2020, down from 6% in the previous quarter – the slowest pace since the financial crisis.

In [ ]:
f, ax = plt.subplots(1,1, figsize=(14,8))
plt.imshow(img)

In [ ]:
covid_forecast=pd.DataFrame(covid_data.groupby(['ObservationDate'],as_index=False)['Confirmed'].sum())
covid_forecast.columns=['ds','y']

model = Prophet()
model.fit(covid_forecast)

future = model.make_future_dataframe(periods=14)
# future.tail()
forecast = model.predict(future)
# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
# fig1 = model.plot(forecast)

## Next 14 days forecast
I have generated a forecast for the next 14 days with the data availble, hover over below chart to see the predicted cases.

In [ ]:
from fbprophet.plot import plot_plotly
import plotly.offline as py
py.init_notebook_mode()
fig = plot_plotly(model, forecast) 
py.iplot(fig)

## How to protect yourself
The best way to prevent yourself from contracting the novel coronavirus is to wash your hands often.

* Use soap and water and wash for at least 20 seconds. If you don’t have soap and water, you can also use hand sanitizer with at least 60 percent alcohol.

* Other ways to protect yourself include the following:

* Stay at least 6 feet away from anyone who seems sick, and avoid large groups of people.
* Avoid touching your face.
* Don’t share personal items with others. This includes things like drinking glasses, utensils, toothbrushes, and lip balm.
* Wipe down high-touch surfaces like doorknobs, keyboards, and stair rails in your home with household cleaners or a diluted bleach solution.
* Wash your hands or use a hand sanitizer after touching surfaces like elevator or ATM buttons, gas pump handles, and grocery carts.
* Stay home and call your doctor if you start having respiratory issues and think your symptoms are consistent with those of COVID-19.

## Conclusion
The forecast was generated on a sequence of number with increasing slope and tends to increase, but, with the measures taken by global nations and citizens taking necessary precautions can contain and eliminate the virus. It's everyone's responsibility to cooperate with the authorities and adhere to the guidelines put in place; can help restore the situation to normalcy.

#### References
* [Corono Meter](https://www.worldometers.info/coronavirus/)
* [Corono Dashboard](https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6)
* [Health Line](https://www.healthline.com/health/coronavirus-incubation-period#how-does-it-spread)
* [FB Prophet](https://pypi.org/project/fbprophet/)
* [World Economic Forum](https://www.weforum.org/focus/coronavirus-covid-19)



In [ ]:
# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Hide/See raw code."></form>''')